# Introduction to tf-shell

To get started, `pip install tf-shell`. tf-shell has a few modules, the one used
in this notebook is `tf_shell`.

In [2]:
import tf_shell

First set up some parameters for the BGV encryption scheme and generate a secret
key.

- `log_n` coresponds to the ring degree.
- `main_moduli` are a product of primes, less than 60 bits, that define the main
modulus. tf-shell uses RNS representation over each of these moduli.
- `scaling_factor` is used to encode fractional numbers, e.g. the number 1.5
with a scaling factor of 2 will be represented as 3.
- `mul_depth_supported` is for debugging purposes. The other parameters are
highly sensitive to the multiplication depth of the computation and tf-shell
will check that the multiplication depth is not exceeded during the computation.

In [3]:
context = tf_shell.create_context64(
    log_n=10,
    main_moduli=[8556589057, 8388812801],
    plaintext_modulus=40961,
    scaling_factor=3,
    mul_depth_supported=3,
    seed="test_seed",
)

secret_key = tf_shell.create_key64(context)

Next, encrypt something. tf-shell currently uses batch-axis packing, meaning the
outer dimension of all tensors is the packing dimension of the BGV scheme. This
is for efficiency purposes and, in short, means the first dimension of all
tensors is 2^log_n defined above.

The cost of operating on a ciphertext (e.g. addition), regardless of how "full"
the first dimension is, is the same.

In [4]:
import tensorflow as tf
tf_data = tf.random.uniform([context.num_slots, 2], dtype=tf.float32, maxval=10)
print(f"The first 3 elements of the data are {tf_data[:3, 0]}")

enc = tf_shell.to_encrypted(tf_data, secret_key, context)

The first 3 elements of the data are [0.62601924 4.461747   5.8008575 ]


Next, perform some homomorphic operations on the ciphertexts. tf-shell can
perform addition, subtraction, and multiplication on ciphertexts, shell
plaintexts, and tensorflow tensors.

In [5]:
# Ciphertext . scalar
ct_scalar_add = enc + 3
ct_scalar_mul = enc * 3

# Ciphertext . tensorflow tensor
three = tf.constant(3, dtype=tf.float32)
ct_tf_add = enc + three
ct_tf_mul = enc * three

# Ciphertext . plaintext
three_repeated = tf.ones([context.num_slots, 1], dtype=tf.float32) * 3
three_repeated = tf_shell.to_shell_plaintext(three_repeated, context)
ct_pt_add = enc + three_repeated
ct_pt_mul = enc * three_repeated

# ciphertext . ciphertext
ct_ct_add = enc + enc
ct_ct_mul = enc * enc

Finally, let's decrypt the results using the secret key.

In [6]:
dec = tf_shell.to_tensorflow(enc, secret_key)
add = tf_shell.to_tensorflow(ct_ct_add, secret_key)
mul = tf_shell.to_tensorflow(ct_ct_mul, secret_key)

print(f"enc:       {dec[:3, 0]}")
print(f"enc + enc: {add[:3, 0]}")
print(f"enc * enc: {mul[:3, 0]}")

enc:       [0.6666667 4.3333335 5.6666665]
enc + enc: [ 1.3333334  8.666667  11.333333 ]
enc * enc: [ 0.44444445 18.777779   32.11111   ]
